# Query Log Analysis

The goal of this notebook is to serve as a way to quickly analyze a query log, based on the following aspects:

- Query-level
- Topic-level
- Session-level
- Term-level
- Entity-level

Our main focus is on understanding how people search, in order to improve entity-oriented search, thus we will tackle these aspects always around the entities.

# Dataset

While we take advantage of the Wikipedia query log for this analysis, we also provide an intermediate format, so that you can convert your own query log into our format and analyze it. Just fill missing columns with NA.